In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [20]:
# JAVA_HOME 환경변수 설정하기
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [21]:
import pandas as pd
import numpy as np
import random
import re, string
from konlpy.tag import Komoran
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [22]:
komoran = Komoran()

In [23]:
train = pd.read_excel('/content/drive/MyDrive/temp/Newsletter_Board/데이터_2022.05.18.xlsx')
train2 = train.copy()

In [24]:
train.head()

,Title,url,thumbnail
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...


### 불용어 제거
한글은 따로 불용어 사전이 존재하지 않기에 불용어 사전을 직접 정의해줘야 함 -> 일반적으로 접속사나 조사 등을 불용어로 정의  
기본 불용어는 https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt 를 참고하고, 이후 단어 빈도수를 확인한 뒤에 불용어를 추가 정의해주는 방향
- 데이터, 빅데이터는 주제 자체이기에 불용어 사전에 추가하여 키워드 추출에서 제외시킴

In [25]:
with open('/content/drive/MyDrive/temp/Newsletter_Board/stopwords.txt', 'r', encoding='cp949') as f:
  list_file = f.readlines() 
stopwords = list_file[0].split(",")
stopwords.append('데이터')
stopwords.append('빅데이터')

In [41]:
# 정규화
def preprocess(text):
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text


# 명사/영단어 추출, 한글자 제외, 불용어 제거
def final(text):
    n = []
    word = komoran.nouns(text)
    p = komoran.pos(text)
    for pos in p:
      if pos[1] in ['SL']:
        word.append(pos[0])
    for w in word:
      if len(w)>1 and w not in stopwords:
        n.append(w)
    return " ".join(n)

# 최종
def finalpreprocess(text):
  return final(preprocess(text))

In [42]:
train['noun'] = train['Title'].apply(lambda x: finalpreprocess(x))

In [43]:
train

,Title,url,thumbnail,noun
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...,과학 방역 중대 전문가 위원회 플랫폼 구성
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...,넥슨 가명 정보 결합 사이언스 강화
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...,인공위성 세종 한글과컴퓨터 이미지
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...,넥슨 가명 정보 결합 사이언스 강화
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...,스페이스 서비스 진출 글로벌 브랜드 KT SAT
...,...,...,...,...
2616,﻿[2022지방선거] ﻿1400만 경기도민 표심 잡아라…경기도지사 후보 김동연 vs...,http://www.smartfn.co.kr/view.php?ud=202205181...,https://search.pstatic.net/common/?src=https%3...,지방 선거 경기도민 표심 아라 경기도지사 후보 김동연 김은혜 vs
2617,"[사설] 가팔라질 기준금리 인상, 부채위기 확산 막아야",https://www.etoday.co.kr/news/view/2134715,,사설 기준 금리 인상 부채 위기 확산
2618,﻿[2022지방선거] 세종시장 후보 이춘희 “행정수도 완성” vs 최민호 “윤석열 ...,http://www.smartfn.co.kr/view.php?ud=202205181...,https://search.pstatic.net/common/?src=https%3...,지방 선거 세종 시장 후보 이춘희 행정 수도 완성 최민호 윤석열 정부 시작 vs
2619,"뉴욕증시 비트코인 흔들 제롬파월 연설 ""자이언트 빅스텝"" 소매판매 0.9%",http://www.g-enews.com/ko-kr/news/article/news...,https://search.pstatic.net/common/?src=https%3...,뉴욕 증시 비트코인 제롬 파월 연설 자이언트 스텝 소매 판매
